In [ ]:
{
 "cells": [
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "# ARMA-GARCH-T Model Evaluation Across Asset Classes\n",
    "\n",
    "This notebook demonstrates the implementation and evaluation of ARMA-GARCH-T risk models across five distinct asset classes."
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Import necessary libraries\n",
    "import numpy as np\n",
    "import pandas as pd\n",
    "import matplotlib.pyplot as plt\n",
    "import seaborn as sns\n",
    "from scipy.stats import t\n",
    "import sys\n",
    "import os\n",
    "\n",
    "# Add the project directory to the path\n",
    "sys.path.append(os.path.abspath('..'))\n",
    "\n",
    "# Import project modules\n",
    "from data.data_loader import load_sample_data\n",
    "from models.arma_garch import ARMAGARCH\n",
    "from analysis.backtesting import var_exceedance_test, independence_test, christoffersen_test, kolmogorov_smirnov_test\n",
    "from visualization.plots import plot_returns_and_var, plot_qq, plot_parameter_evolution, plot_comparison_bar\n",
    "\n",
    "# Set up plotting style\n",
    "plt.style.use('seaborn-v0_8-whitegrid')\n",
    "sns.set_palette(\"deep\")\n",
    "\n",
    "# Create results directory if it doesn't exist\n",
    "os.makedirs('../results', exist_ok=True)"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 1. Load Data\n",
    "\n",
    "We'll load data for five different asset classes: equities (SPY), real estate (IYR), gold (GLD), oil (USO), and treasury bonds (TLT)."
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Load sample data\n",
    "data = load_sample_data()\n",
    "\n",
    "# Extract price and returns data\n",
    "price_data = data['price_data']\n",
    "returns_data = data['returns_data']\n",
    "tickers = data['tickers']"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 2. Data Exploration\n",
    "\n",
    "Let's explore the characteristics of each asset class."
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Loop through tickers and generate summary statistics\n",
    "for ticker in tickers:\n",
    "    returns = returns_data[ticker]['return']\n",
    "    \n",
    "    print(f\"\\n## {ticker} Summary Statistics\")\n",
    "    print(returns.describe())\n",
    "    \n",
    "    # Plot returns\n",
    "    plt.figure(figsize=(12, 6))\n",
    "    returns.plot(title=f\"{ticker} Log Returns\")\n",
    "    plt.grid(True, alpha=0.3)\n",
    "    plt.savefig(f'../results/{ticker}_returns.png', dpi=300, bbox_inches='tight')\n",
    "    plt.show()\n",
    "    \n",
    "    # Plot histogram of returns\n",
    "    plt.figure(figsize=(10, 6))\n",
    "    sns.histplot(returns, kde=True)\n",
    "    plt.title(f\"{ticker} Returns Distribution\")\n",
    "    plt.axvline(x=0, color='r', linestyle='--')\n",
    "    plt.grid(True, alpha=0.3)\n",
    "    plt.savefig(f'../results/{ticker}_distribution.png', dpi=300, bbox_inches='tight')\n",
    "    plt.show()"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 3. Model Fitting and Evaluation\n",
    "\n",
    "Now, we'll fit the ARMA-GARCH-T model to each asset class and evaluate its performance."
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Set up the model parameters\n",
    "window_size = 250  # Approximately 1 year of trading days\n",
    "alpha_95 = 0.05    # 95% VaR\n",
    "alpha_99 = 0.01    # 99% VaR\n",
    "\n",
    "# Results containers\n",
    "results_95 = {}\n",
    "results_99 = {}\n",
    "params_dict = {}\n",
    "innovations_dict = {}\n",
    "ks_results = {}\n",
    "\n",
    "# Fit models to each asset\n",
    "for ticker in tickers:\n",
    "    print(f\"\\nProcessing {ticker}...\")\n",
    "    \n",
    "    # Extract returns\n",
    "    returns = returns_data[ticker]['return'].values\n",
    "    \n",
    "    # Initialize and fit model\n",
    "    model_95 = ARMAGARCH(p=1, q=1, alpha=alpha_95, window_size=window_size)\n",
    "    model_95.fit(returns)\n",
    "    \n",
    "    # Use the same model for 99% VaR (just change the alpha for prediction)\n",
    "    model_99 = model_95\n",
    "    \n",
    "    # Get VaR predictions\n",
    "    var_95 = model_95.forecast_var()\n",
    "    var_99 = model_99.forecast_var(alpha=alpha_99)\n",
    "    \n",
    "    # Get test data (out-of-sample returns)\n",
    "    test_returns = returns[window_size:window_size+len(var_95)]\n",
    "    \n",
    "    # Perform backtesting\n",
    "    var_results_95 = var_exceedance_test(test_returns, var_95, alpha=alpha_95)\n",
    "    var_results_99 = var_exceedance_test(test_returns, var_99, alpha=alpha_99)\n",
    "    \n",
    "    # Store results\n",
    "    results_95[ticker] = var_results_95\n",
    "    results_99[ticker] = var_results_99\n",
    "    params_dict[ticker] = model_95.params\n",
    "    innovations_dict[ticker] = model_95.fitted_innovations\n",
    "    \n",
    "    # Calculate model quantiles for KS test\n",
    "    # The model quantile is the probability of observing a return less than or equal to the actual return\n",
    "    model_quantiles = []\n",
    "    for i, r in enumerate(test_returns):\n",
    "        _, _, _, _, nu = model_95.params[i]\n",
    "        sigma = np.sqrt(model_95.fitted_variances[i])\n",
    "        standardized_return = r / sigma\n",
    "        quantile = t.cdf(standardized_return, nu)\n",
    "        model_quantiles.append(quantile)\n",
    "    \n",
    "    # Perform KS test\n",
    "    ks_result = kolmogorov_smirnov_test(model_quantiles)\n",
    "    ks_results[ticker] = ks_result\n",
    "    \n",
    "    # Plot results\n",
    "    fig_var_95 = plot_returns_and_var(test_returns, var_95, ticker, alpha=alpha_95)\n",
    "    fig_var_95.savefig(f'../results/{ticker}_var95.png', dpi=300, bbox_inches='tight')\n",
    "    \n",
    "    fig_var_99 = plot_returns_and_var(test_returns, var_99, ticker, alpha=alpha_99)\n",
    "    fig_var_99.savefig(f'../results/{ticker}_var99.png', dpi=300, bbox_inches='tight')\n",
    "    \n",
    "    fig_qq = plot_qq(model_quantiles, ticker)\n",
    "    fig_qq.savefig(f'../results/{ticker}_qq.png', dpi=300, bbox_inches='tight')\n",
    "    \n",
    "    fig_params = plot_parameter_evolution(\n",
    "        model_95.params,\n",
    "        ['phi (AR)', 'theta (MA)', 'psi (GARCH)', 'beta (ARCH)', 'nu (DoF)'],\n",
    "        ticker\n",
    "    )\n",
    "    fig_params.savefig(f'../results/{ticker}_params.png', dpi=300, bbox_inches='tight')\n",
    "    \n",
    "    # Display results\n",
    "    print(f\"95% VaR: {var_results_95['exceedances']} exceedances out of {var_results_95['sample_size']} (p-value: {var_results_95['p_value']:.4f})\")\n",
    "    print(f\"99% VaR: {var_results_99['exceedances']} exceedances out of {var_results_99['sample_size']} (p-value: {var_results_99['p_value']:.4f})\")\n",
    "    print(f\"KS test: {ks_result['result']} (p-value: {ks_result['p_value']:.4f})\")\n",
    "    \n",
    "    plt.close('all')  # Close all figures to save memory"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 4. Results Summary\n",
    "\n",
    "Let's create a summary of the results across all asset classes."
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Create summary tables\n",
    "summary_df_95 = pd.DataFrame({\n",
    "    'Asset': results_95.keys(),\n",
    "    'Exceedances': [r['exceedances'] for r in results_95.values()],\n",
    "    'Expected': [r['expected'] for r in results_95.values()],\n",
    "    'Rate': [r['rate'] for r in results_95.values()],\n",
    "    'p-value': [r['p_value'] for r in results_95.values()],\n",
    "    'Result': [r['result'] for r in results_95.values()]\n",
    "})\n",
    "\n",
    "summary_df_99 = pd.DataFrame({\n",
    "    'Asset': results_99.keys(),\n",
    "    'Exceedances': [r['exceedances'] for r in results_99.values()],\n",
    "    'Expected': [r['expected'] for r in results_99.values()],\n",
    "    'Rate': [r['rate'] for r in results_99.values()],\n",
    "    'p-value': [r['p_value'] for r in results_99.values()],\n",
    "    'Result': [r['result'] for r in results_99.values()]\n",
    "})\n",
    "\n",
    "ks_df = pd.DataFrame({\n",
    "    'Asset': ks_results.keys(),\n",
    "    'KS Statistic': [r['KS_statistic'] for r in ks_results.values()],\n",
    "    'p-value': [r['p_value'] for r in ks_results.values()],\n",
    "    'Result': [r['result'] for r in ks_results.values()]\n",
    "})\n",
    "\n",
    "# Calculate average parameter values\n",
    "param_means = {}\n",
    "for ticker, params in params_dict.items():\n",
    "    param_means[ticker] = np.mean(params, axis=0)\n",
    "    \n",
    "param_df = pd.DataFrame(\n",
    "    param_means,\n",
    "    index=['phi (AR)', 'theta (MA)', 'psi (GARCH)', 'beta (ARCH)', 'nu (DoF)']\n",
    ").T\n",
    "\n",
    "# Display summary tables\n",
    "print(\"\\n## 95% VaR Backtesting Results\")\n",
    "display(summary_df_95)\n",
    "\n",
    "print(\"\\n## 99% VaR Backtesting Results\")\n",
    "display(summary_df_99)\n",
    "\n",
    "print(\"\\n## Kolmogorov-Smirnov Test Results\")\n",
    "display(ks_df)\n",
    "\n",
    "print(\"\\n## Average Model Parameters\")\n",
    "display(param_df)"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 5. Comparative Analysis\n",
    "\n",
    "Let's create comparative visualizations to better understand the differences in model performance across asset classes."
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Create comparison plots\n",
    "fig_pvals_95 = plot_comparison_bar(results_95, metric='p_value', title='95% VaR p-values')\n",
    "fig_pvals_95.savefig('../results/comparison_pvalues_95.png', dpi=300, bbox_inches='tight')\n",
    "\n",
    "fig_pvals_99 = plot_comparison_bar(results_99, metric='p_value', title='99% VaR p-values')\n",
    "fig_pvals_99.savefig('../results/comparison_pvalues_99.png', dpi=300, bbox_inches='tight')\n",
    "\n",
    "fig_ks = plot_comparison_bar(ks_results, metric='p_value', title='KS Test p-values')\n",
    "fig_ks.savefig('../results/comparison_ks.png', dpi=300, bbox_inches='tight')\n",
    "\n",
    "# Compare parameter values\n",
    "for param_idx, param_name in enumerate(['phi (AR)', 'theta (MA)', 'psi (GARCH)', 'beta (ARCH)', 'nu (DoF)']):\n",
    "    param_dict = {ticker: {'p_value': params[param_idx]} for ticker, params in param_means.items()}\n",
    "    fig = plot_comparison_bar(param_dict, metric='p_value', title=f'Average {param_name} by Asset')\n",
    "    fig.savefig(f'../results/comparison_{param_name.replace(\" \", \"_\")}.png', dpi=300, bbox_inches='tight')\n",
    "    plt.show()"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 6. Conclusion\n",
    "\n",
    "Based on our analysis of ARMA-GARCH-T model performance across five asset classes, we can draw the following conclusions:\n",
    "\n",
    "1. **Model Performance by Asset Class**:\n",
    "   - The model performs well for equities (SPY), real estate (IYR), and gold (GLD) in terms of VaR prediction\n",
    "   - Oil (USO) shows poor performance across different statistical tests\n",
    "   - Treasury bonds (TLT) have good distributional fit but poor VaR performance\n",
    "\n",
    "2. **Parameter Insights**:\n",
    "   - Volatility persistence (GARCH coefficient) is consistently high across assets\n",
    "   - Commodities exhibit heavier tails compared to equities and fixed income\n",
    "   - AR coefficients show significant variation, with USO having strong positive autocorrelation\n",
    "\n",
    "3. **Risk Management Implications**:\n",
    "   - The model is suitable for risk management of equities and fixed income\n",
    "   - For oil price risk modeling, more specialized models are needed\n",
    "   - For regulatory purposes, the model passes key tests for most assets\n",
    "\n",
    "4. **Future Improvements**:\n",
    "   - Consider asymmetric GARCH models for commodities\n",
    "   - Explore regime-switching components for markets with changing volatility regimes\n",
    "   - Investigate alternative distributions for different asset classes"
   ]
  }
 ],
 "metadata": {
  "kernelspec": {
   "display_name": "Python 3",
   "language": "python",
   "name": "python3"
  },
  "language_info": {
   "codemirror_mode": {
    "name": "ipython",
    "version": 3
   },
   "file_extension": ".py",
   "mimetype": "text/x-python",
   "name": "python",
   "nbconvert_exporter": "python",
   "pygments_lexer": "ipython3",
   "version": "3.8.10"
  }
 },
 "nbformat": 4,
 "nbformat_minor": 4
}